In [2]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
pd.options.display.max_columns = 100
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
pd.options.display.max_rows = 100
from itertools import islice
import urllib2
from bs4 import BeautifulSoup
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from DatumBox import DatumBox
import re
from urlparse import urlparse
import seaborn as sns # To have more than seven colors in pie diagram
import plotly.plotly as py # For Gantt Chart
import plotly.figure_factory as ff # For Gantt Chart
import sklearn

In [3]:
import urllib
from bs4 import BeautifulSoup
import csv
import urllib2
import re
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from itertools import chain
from nltk.corpus import stopwords
from DatumBox import DatumBox
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import os.path as osp
import os
datum_box = DatumBox("2a13913dda346761765020c1f66e34f8")
TAG_RE = re.compile(r"<[^>]+>")
nltk.download('punkt')
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
cachedStopWords = stopwords.words("english")

def remove_tags(text):
    return TAG_RE.sub('', text)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

def main_text(url):
    file_name = url.replace('/','#')
    if len(file_name) <= 255:
        #If the file is already there
        if osp.isfile(os.path.abspath(os.curdir)+"/Websites/"+file_name):
            f = open(os.path.abspath(os.curdir)+"/Websites/"+file_name,'r')
            #text contains the text from a webpage that are stored locally
            text = f.read()
            f.close()
        #If the file isnt there then text is extracted with beautiful soup and stored locally
        else:
            html = urllib.urlopen(url).read()
            soup = BeautifulSoup(html,"html.parser")

            # kill all script and style elements
            for script in soup(["script", "style"]):
                script.extract()    # rip it out

            # get text
            text = soup.get_text()

            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = '\n'.join(chunk for chunk in chunks if chunk)
            text = ' '.join([word for word in text.split() if word not in cachedStopWords])
            for char in text:
                if(not((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90))):
                    text = text.replace (char," ")
            f = open(os.path.abspath(os.curdir)+'/Websites/'+file_name,'w')
            f.write(text)
            f.close()
            print "/Seen First Time/",
    else:
        html = urllib.urlopen(url).read()
        soup = BeautifulSoup(html,"html.parser")
        print "/Cant save in file/",
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        text = ' '.join([word for word in text.split() if word not in cachedStopWords])
        for char in text:
            if(not((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90))):
                text = text.replace (char," ")
    return text

def openfile(f):
    x = []
    y = []
    finial = []; i=2
    csvfile = open(f,'rb')
    reader = csv.DictReader(csvfile)
    text1='';text2='';text1_main='';text2_main=''
    synonyms=None
    for row in reader:
        synonyms=None
        if i==0:
            print str(round(cosine_sim(text1,text2),2))+ "      " +str(round(cosine_sim(text1_main,text2_main),2))
            i=2
            text1='';text1_main=''
            text2='';text2_main=''
        elif i==1:
            text1 = main_text(row['url'])
            text1_main = text1 + ' '
            for j in text1.split(' '):
                synonyms = wordnet.synsets(j)
                synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
                for k in synonyms:
                    text1_main += k + ' '
            i-=1
        elif i==2:
            text2 = main_text(row['url'])
            text2_main = text2 + ' '
            for j in text2.split(' '):
                synonyms = wordnet.synsets(j)
                synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
                for k in synonyms:
                    text2_main += k + ' '
            i-=1
    print a
    csvfile.close()

#openfile('/home/chaitanya/Documents/Codes/IR_Vidhya mam/resources/input/10.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/sreeram_0xb5e/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def wordnet_similarity(a,b):
    url1 = a
    url2 = b
    text1='';text2='';text1_main='';text2_main=''
    text1 = main_text(url1)
    text2 = main_text(url2)
    word_count = 0;similar = 0
    text1_split = text1.split(' ');text2_split = text2.split(' ')
    text1_split = [x.upper() for x in text1_split if x]
    text2_split = [x.upper() for x in text2_split if x]
    similar = 0.0;word_count = 0.0;flag=0
    
    for j in text1_split:
        flag = 0
        if j not in text2_split:
            synonyms = wordnet.synsets(j)
            synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
            for k in synonyms:
                if k in text2_split:
                    flag = 1
        else:
            similar += 1
        if flag == 1:
            similar += 1
        word_count += 1

    for j in text2_split:
        flag = 0
        if j not in text1_split:
            synonyms = wordnet.synsets(j)
            synonyms = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
            for k in synonyms:
                if k in text1_split:
                    flag = 1
        else:
            similar += 1
        if flag == 1:
            similar += 1
        word_count += 1

    #text2_main += k + ' '
    if word_count == 0:
        return 0
    return similar/word_count


In [5]:
disp_data= pd.read_csv('internal_cache.csv')
disp_data = disp_data[~disp_data["url"].str.contains("google")]
disp_data = disp_data[~disp_data["url"].str.contains("home")]
i_range = 100
j_range = 130
test_sample = disp_data[i_range:j_range]
#test_sample

In [1]:
import urllib2
import obo

def find_buffer(url):
    text_in_all_urls = []
    text = main_text(url)
    fullwordlist = obo.stripNonAlphaNum(text)
    wordlist = obo.removeStopwords(fullwordlist, obo.stopwords)
    dictionary = obo.wordListToFreqDict(wordlist)
    sorteddict = obo.sortFreqDict(dictionary)
    for i in sorteddict[:30]:
        text_in_all_urls.append(i[1])
    return text_in_all_urls


#print find_buffer("https://journalistsresource.org/studies/society/social-media/multitasking-social-media-distraction-what-does-research-say")

In [3]:
print "\n\nNow the commmon most reccuring words\n"
main_text = []
final = find_buffer("https://www.w3schools.com/Js/")

for i in final:
    for j in i:
        main_text.append(j)
main_text = obo.wordListToFreqDict(main_text[:])
main_text = obo.sortFreqDict(main_text)
for i in main_text[:10]:
    print i[1],



Now the commmon most reccuring words



TypeError: 'list' object is not callable

In [63]:
clusters = []
cluster_buffer = []
thres = 0.2

for i in range(i_range,j_range):
    curr_url = disp_data.iloc[i]['url']
    
    if (len(clusters) == 0):
        clusters.append(list(str(temp)))
        cluster_buffer.append([])
        
    if(len(clusters) > 0):
        c_indx = 0
        max_sim = 0
        
        for c in range(0,len(clusters)):
            if (len(clusters[c]) < 2):
                ws = wordnet_similarity(clusters[i],curr_url);
            else:
                ws = wordnet_similarity_url_text(curr_url,cluster_buffer)

            if (ws > max_sim):
                    max_sim = ws
                    c_indx = c
        
        if (max_sim > thres):
            clusters[c].append(curr_url)
            #Cluster buffer case
            
            #When there are strings in the buffer
            if (len(cluster_buffer[i]) > 2):
            
            #When the buffer is empty
            else:
            
        else:
            clusters.append(list(str(curr_url)))
            cluster_buffer.append([])

{'The', 'Uber'}